### 6. Queremos saber cuánto pesaría si quisiéramos bajar todas las apps de un género, para todos los géneros. Para eso se pide: Calcular separado por géneros, cuanto pesarian todas las apps que tienen ese género (Tener en cuenta que si una app tiene acción y arte, su peso cuenta para ambos géneros) ⭐⭐

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [2]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
dataframe = sqlContext.read.csv('googleplaystore.csv', header=True)
apps = dataframe.rdd
apps.take(5)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/09 14:19:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating='4.1', Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up'),
 Row(App='Coloring book moana', Category='ART_AND_DESIGN', Rating='3.9', Reviews='967', Size='14M', Installs='500,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design;Pretend Play', Last Updated='January 15, 2018', Current Ver='2.0.0', Android Ver='4.0.3 and up'),
 Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps', Category='ART_AND_DESIGN', Rating='4.7', Reviews='87510', Size='8.7M', Installs='5,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='August 1, 2018', Current Ver='1.2.4', Android Ver='4.0.3 and up'),
 Row(App='Sketch - Draw & Paint', Category='ART_AND_DESIGN', Rating='4.5', Reviews='215644', S

In [5]:
apps.map(lambda x: x.Size[-1]).distinct().collect()

['M', 'e', 'k', 'N', 'S', '+']

Los tamaños de las aplicaciones vienen en mega y kilobytes, voy a filtrar todas las apps cuyo "Size" no termine en "M" o en "k" y hago la conversión para dejar todo expresado en megabytes (1k = 0.001M)

In [4]:
apps.filter(lambda x: x.Size[-1] in ['M', 'k']).map(lambda x: (x.Genres.split(';'), float(x.Size[:-1]) if x.Size[-1] == 'M' else float(x.Size[:-1]) * 0.001))\
.flatMap(lambda x: [(genre, x[1]) for genre in x[0]]).reduceByKey(lambda a, b: a + b).collect()

[('Art & Design', 812.0000000000002),
 ('Pretend Play', 4082.2000000000003),
 ('Creativity', 1236.9),
 ('Action & Adventure', 5757.099999999999),
 ('Auto & Vehicles', 1502.786),
 ('Beauty', 648.4000000000001),
 ('Books & Reference', 2740.232),
 ('Business', 5788.865),
 ('Comics', 687.7530000000002),
 ('Communication', 3118.176999999999),
 ('Dating', 3524.6180000000018),
 ('Education', 14758.522999999994),
 ('Brain Games', 1967.6999999999998),
 ('Entertainment', 9057.860000000002),
 ('Music & Video', 865.3000000000001),
 ('Events', 795.934),
 ('Finance', 5504.069999999997),
 ('Food & Drink', 2218.5),
 ('Health & Fitness', 6144.072000000001),
 ('House & Home', 1086.5559999999998),
 ('Libraries & Demo', 908.8335),
 ('Lifestyle', 5090.302000000001),
 ('Adventure', 3411.4),
 ('Arcade', 8858.800000000001),
 ('Casual', 8482.000000000004),
 ('Card', 1348.516),
 ('Strategy', 5258.900000000001),
 ('Action', 17347.343),
 ('Puzzle', 5307.6500000000015),
 ('Sports', 8652.433000000003),
 ('Word', 89